In [1]:
# dependencies
import pandas as pd

In [12]:
# read excel files into dataframes

season = "2016_2017"

file_to_load = f"Resources/{season}.xlsx"
df = pd.read_excel(file_to_load)
df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Alex Abrines\abrinal01,2.0,5.0,0.393,1.4,0.6,0.7,0.898,1.3,0.6,0.5,0.1,0.5,6.0,-0.1
1,Quincy Acy\acyqu01,1.8,4.5,0.412,1.0,1.2,1.6,0.750,3.0,0.5,0.4,0.4,0.6,5.8,-0.1
2,Quincy Acy\acyqu01,0.8,2.8,0.294,0.2,0.3,0.5,0.667,1.3,0.0,0.0,0.0,0.3,2.2,-0.2
3,Quincy Acy\acyqu01,2.0,4.8,0.425,1.1,1.3,1.8,0.754,3.3,0.6,0.4,0.5,0.6,6.5,0.0
4,Steven Adams\adamsst01,4.7,8.2,0.571,0.0,2.0,3.2,0.611,7.7,1.1,1.1,1.0,1.8,11.3,1.5


In [13]:
# enter roster size and number of teams in fantasy league
roster_size = 13
num_teams = 10
sample_size = roster_size*num_teams

In [14]:
# clean player names
df["Player"] = df["Player"].str.split("\\", n = 1, expand = True)

# rename column headers
df = df.rename(columns={"PS/G": "PTS", "FG%": "FGP", "FT%":"FTP"})

# keep only total season stats for players with more than 1 team
df = df.drop_duplicates(subset=["Player"], keep = 'first')

df.head()

,Player,FG,FGA,FGP,3P,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PTS,VORP
0,Alex Abrines,2.0,5.0,0.393,1.4,0.6,0.7,0.898,1.3,0.6,0.5,0.1,0.5,6.0,-0.1
1,Quincy Acy,1.8,4.5,0.412,1.0,1.2,1.6,0.750,3.0,0.5,0.4,0.4,0.6,5.8,-0.1
4,Steven Adams,4.7,8.2,0.571,0.0,2.0,3.2,0.611,7.7,1.1,1.1,1.0,1.8,11.3,1.5
5,Arron Afflalo,3.0,6.9,0.440,1.0,1.4,1.5,0.892,2.0,1.3,0.3,0.1,0.7,8.4,-0.6
6,Alexis Ajinca,2.3,4.6,0.500,0.0,0.7,1.0,0.725,4.5,0.3,0.5,0.6,0.8,5.3,-0.3


In [15]:
# get initial group of top players using VORP
top_players = df.sort_values("VORP", ascending = False).head(sample_size)
top_players["TOP"] = 1
top_players.head()

,Player,FG,FGA,FGP,3P,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PTS,VORP,TOP
558,Russell Westbrook,10.2,24.0,0.425,2.5,8.8,10.4,0.845,10.7,10.4,1.6,0.4,5.4,31.6,12.4,1
210,James Harden,8.3,18.9,0.440,3.2,9.2,10.9,0.847,8.1,11.2,1.5,0.5,5.7,29.1,9.0,1
269,LeBron James,9.9,18.2,0.548,1.7,4.8,7.2,0.674,8.6,8.7,1.2,0.6,4.1,26.4,7.3,1
19,Giannis Antetokounmpo,8.2,15.7,0.521,0.6,5.9,7.7,0.770,8.8,5.4,1.6,1.9,2.9,22.9,6.9,1
79,Jimmy Butler,7.5,16.5,0.455,1.2,7.7,8.9,0.865,6.2,5.5,1.9,0.4,2.1,23.9,6.3,1


In [16]:
# merge TOP with original df
df = pd.merge(top_players[["Player", "TOP"]], df, on="Player", how="outer").fillna(0)
df.head()

,Player,TOP,FG,FGA,FGP,3P,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PTS,VORP
0,Russell Westbrook,1.0,10.2,24.0,0.425,2.5,8.8,10.4,0.845,10.7,10.4,1.6,0.4,5.4,31.6,12.4
1,James Harden,1.0,8.3,18.9,0.440,3.2,9.2,10.9,0.847,8.1,11.2,1.5,0.5,5.7,29.1,9.0
2,LeBron James,1.0,9.9,18.2,0.548,1.7,4.8,7.2,0.674,8.6,8.7,1.2,0.6,4.1,26.4,7.3
3,Giannis Antetokounmpo,1.0,8.2,15.7,0.521,0.6,5.9,7.7,0.770,8.8,5.4,1.6,1.9,2.9,22.9,6.9
4,Jimmy Butler,1.0,7.5,16.5,0.455,1.2,7.7,8.9,0.865,6.2,5.5,1.9,0.4,2.1,23.9,6.3


In [17]:
# calculate adjusted percentages
top_players['adj_FG'] = (10*(top_players['FG'].sum() / sample_size) + top_players["FG"]) / (10*(top_players['FGA'].sum() / sample_size) + top_players["FGA"])
top_players['adj_FT'] = (10*(top_players['FT'].sum() / sample_size) + top_players["FT"]) / (10*(top_players['FTA'].sum() / sample_size) + top_players["FTA"])
df['adj_FG'] = (10*(top_players['FG'].sum() / sample_size) + df["FG"]) / (10*(top_players['FGA'].sum() / sample_size) + df["FGA"])
df['adj_FT'] = (10*(top_players['FT'].sum() / sample_size) + df["FT"]) / (10*(top_players['FTA'].sum() / sample_size) + df["FTA"])
df.head()

,Player,TOP,FG,FGA,FGP,3P,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PTS,VORP,adj_FG,adj_FT
0,Russell Westbrook,1.0,10.2,24.0,0.425,2.5,8.8,10.4,0.845,10.7,10.4,1.6,0.4,5.4,31.6,12.4,0.465935,0.796967
1,James Harden,1.0,8.3,18.9,0.440,3.2,9.2,10.9,0.847,8.1,11.2,1.5,0.5,5.7,29.1,9.0,0.469635,0.797000
2,LeBron James,1.0,9.9,18.2,0.548,1.7,4.8,7.2,0.674,8.6,8.7,1.2,0.6,4.1,26.4,7.3,0.484705,0.761975
3,Giannis Antetokounmpo,1.0,8.2,15.7,0.521,0.6,5.9,7.7,0.770,8.8,5.4,1.6,1.9,2.9,22.9,6.9,0.480814,0.779123
4,Jimmy Butler,1.0,7.5,16.5,0.455,1.2,7.7,8.9,0.865,6.2,5.5,1.9,0.4,2.1,23.9,6.3,0.472226,0.799180


In [18]:
# calculate z-scores and average z-score
df["zFG"] = (df["adj_FG"] - top_players["adj_FG"].mean()) / top_players["adj_FG"].std()
df["zFT"] = (df["adj_FT"] - top_players["adj_FT"].mean()) / top_players["adj_FT"].std()
df["z3P"] = (df["3P"] - top_players["3P"].mean()) / top_players["3P"].std()
df["zPTS"] = (df["PTS"] - top_players["PTS"].mean()) / top_players["PTS"].std()
df["zREB"] = (df["TRB"] - top_players["TRB"].mean()) / top_players["TRB"].std()
df["zAST"] = (df["AST"] - top_players["AST"].mean()) / top_players["AST"].std()
df["zSTL"] = (df["STL"] - top_players["STL"].mean()) / top_players["STL"].std()
df["zBLK"] = (df["BLK"] - top_players["BLK"].mean()) / top_players["BLK"].std()
df["zTOV"] = (top_players["TOV"].mean() - df["TOV"]) / top_players["TOV"].std()
df["zAVG"] = (df["zFG"] + df["zFT"] + df["z3P"] + df["zPTS"] + df["zREB"] + df["zAST"] + df["zSTL"] + df["zBLK"] + df["zTOV"]) / 9
df.head()

,Player,TOP,FG,FGA,FGP,3P,FT,FTA,FTP,TRB,...,zFG,zFT,z3P,zPTS,zREB,zAST,zSTL,zBLK,zTOV,zAVG
0,Russell Westbrook,1.0,10.2,24.0,0.425,2.5,8.8,10.4,0.845,10.7,...,-1.933527,1.453438,1.400405,2.642864,1.802251,3.150813,1.438198,-0.557937,-4.005890,0.598957
1,James Harden,1.0,8.3,18.9,0.440,3.2,9.2,10.9,0.847,8.1,...,-1.136879,1.456668,2.157606,2.260811,0.829772,3.496581,1.180314,-0.369052,-4.333210,0.615846
2,LeBron James,1.0,9.9,18.2,0.548,1.7,4.8,7.2,0.674,8.6,...,2.106966,-1.909006,0.535033,1.848194,1.016787,2.416055,0.406663,-0.180167,-2.587505,0.405891
3,Giannis Antetokounmpo,1.0,8.2,15.7,0.521,0.6,5.9,7.7,0.770,8.8,...,1.269468,-0.261218,-0.654854,1.313321,1.091593,0.989762,1.438198,2.275337,-1.278226,0.687042
4,Jimmy Butler,1.0,7.5,16.5,0.455,1.2,7.7,8.9,0.865,6.2,...,-0.579300,1.666098,-0.005825,1.466142,0.119114,1.032983,2.211850,-0.557937,-0.405373,0.549750


In [19]:
# rank by avg z-score
df = df.sort_values("zAVG", ascending = False).reset_index(drop=True)
df.index += 1
df

,Player,TOP,FG,FGA,FGP,3P,FT,FTA,FTP,TRB,...,zFG,zFT,z3P,zPTS,zREB,zAST,zSTL,zBLK,zTOV,zAVG
1,Kevin Durant,1.0,8.9,16.5,0.537,1.9,5.4,6.2,0.875,8.3,...,1.806974,1.328087,0.751376,1.649527,0.904578,0.730435,0.148779,1.708682,-0.514480,0.945995
2,Anthony Davis,1.0,10.3,20.3,0.505,0.5,6.9,8.6,0.802,11.8,...,1.084565,0.409140,-0.763025,2.092708,2.213684,-0.436532,0.664547,2.841991,-0.732693,0.819376
3,Stephen Curry,1.0,8.5,18.3,0.468,4.1,4.1,4.6,0.898,4.5,...,-0.327231,1.261417,3.131150,1.680091,-0.516737,1.508414,1.953966,-0.935707,-1.387332,0.707559
4,Giannis Antetokounmpo,1.0,8.2,15.7,0.521,0.6,5.9,7.7,0.770,8.8,...,1.269468,-0.261218,-0.654854,1.313321,1.091593,0.989762,1.438198,2.275337,-1.278226,0.687042
5,Kawhi Leonard,1.0,8.6,17.7,0.485,2.0,6.3,7.2,0.880,5.8,...,0.321193,1.570088,0.859548,1.710655,-0.030498,0.168562,1.953966,0.008718,-0.405373,0.684095
6,Karl-Anthony Towns,1.0,9.8,18.0,0.542,1.2,4.3,5.2,0.832,12.3,...,2.101813,0.586403,-0.005825,1.649527,2.400699,-0.177206,-0.882756,1.142027,-0.950906,0.651531
7,James Harden,1.0,8.3,18.9,0.440,3.2,9.2,10.9,0.847,8.1,...,-1.136879,1.456668,2.157606,2.260811,0.829772,3.496581,1.180314,-0.369052,-4.333210,0.615846
8,Chris Paul,1.0,6.1,12.9,0.476,2.0,3.8,4.3,0.892,5.0,...,-0.052931,1.108014,0.859548,0.579780,-0.329722,2.632161,2.469734,-1.124592,-0.732693,0.601033
9,Russell Westbrook,1.0,10.2,24.0,0.425,2.5,8.8,10.4,0.845,10.7,...,-1.933527,1.453438,1.400405,2.642864,1.802251,3.150813,1.438198,-0.557937,-4.005890,0.598957
10,Jimmy Butler,1.0,7.5,16.5,0.455,1.2,7.7,8.9,0.865,6.2,...,-0.579300,1.666098,-0.005825,1.466142,0.119114,1.032983,2.211850,-0.557937,-0.405373,0.549750


In [20]:
# export to excel file
df.to_excel(f"Resources/ranked_{season}.xlsx", index=True)